<a href="https://colab.research.google.com/github/Bhavajna123/Predict-Credit-Card-Approval-ML/blob/main/final_credit_card_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q scikit-learn pandas


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier

# Load dataset
df = pd.read_csv("application_record.csv")

# Target column (1 = Eligible, 0 = Not Eligible)
target_col = "FLAG_OWN_REALTY"

# ID column
id_col = "ID"


In [3]:
# Encode categorical columns
label_encoders = {}
for col in df.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))
    label_encoders[col] = le


In [4]:
# Features and target
X = df.drop(columns=[target_col])
y = df[target_col]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [5]:
# Train model faster with fewer trees and parallel processing
model = RandomForestClassifier(
    random_state=42,
    n_estimators=30,  # reduced from 100 for faster training
    n_jobs=-1          # use all CPU cores
)
model.fit(X_train, y_train)
print("✅ Model trained successfully!")


✅ Model trained successfully!


In [6]:
# ----- ID-based Eligibility Check -----
def check_eligibility_by_id():
    while True:
        try:
            search_val = input("Enter applicant ID (or 0 to exit): ").strip()
            if search_val == "0":
                print("Exiting eligibility check.")
                break
            search_val = int(search_val)
        except ValueError:
            print("⚠️ Please enter a valid numeric ID.")
            continue

        # Find applicant
        person = df[df[id_col] == search_val]
        if person.empty:
            print(f"❌ No record found for ID {search_val}.\n")
            continue

        # Prepare features
        X_person = person.drop(columns=[target_col])

        # Prediction
        pred = model.predict(X_person)[0]
        pred_proba = model.predict_proba(X_person)[0][1]  # probability for class 1

        # Map prediction
        status = "ELIGIBLE ✅" if pred == 1 else "NOT ELIGIBLE ❌"
        print(f"Person with ID {search_val} is {status}")
        print(f"Predicted probability of being eligible: {pred_proba*100:.2f}%\n")

# Run the interactive check
check_eligibility_by_id()


Enter applicant ID (or 0 to exit): 5008804
Person with ID 5008804 is ELIGIBLE ✅
Predicted probability of being eligible: 73.33%

Enter applicant ID (or 0 to exit): 5008876
Person with ID 5008876 is ELIGIBLE ✅
Predicted probability of being eligible: 100.00%

Enter applicant ID (or 0 to exit): 5007644
❌ No record found for ID 5007644.

Enter applicant ID (or 0 to exit): 5008866
Person with ID 5008866 is ELIGIBLE ✅
Predicted probability of being eligible: 100.00%

Enter applicant ID (or 0 to exit): 5453566
Person with ID 5453566 is NOT ELIGIBLE ❌
Predicted probability of being eligible: 0.00%

Enter applicant ID (or 0 to exit): 0
Exiting eligibility check.
